# 빅데이터처리
## Week09_Quiz
### 2016003709 윤가영

In [1]:
# To find out where the pyspark
import findspark
findspark.init()

In [2]:
# Creating Spark Context
from pyspark import SparkContext
sc = SparkContext("local", "first app")

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import copy

#### KFold 모듈 호출

In [4]:
from sklearn.model_selection import KFold

In [5]:
import csv

In [6]:
def create_dataset():
    f = open('./W09/income_evaluation.csv', 'r', encoding='utf-8')
    rdr = csv.reader(f)

    dataset = []
    for line in rdr:
        dataset.append(line)
    f.close()   
    
    return np.array(dataset)

#### NaiveBayes 클래스 생성

In [10]:
class NaiveBayes:

    def find_prob(self, t):
        """
        해당 tuple(key)에 해당하는 probability 반환.
        key가 없거나 확률이 0인경우 매우 작은 수(1e-18) 대체
        """
        result = [0 for _ in range(2)]
        
        if(t not in self.x_y2.keys()):
            result = [1e-18,1e-18]
            return result
        
        prob_list = self.x_y2[t]
        
        for prob in prob_list:
            result[prob[0]] = prob[1]

        for i in range(2):
            if result[i] == 0:
                result[i] = 1e-18
        return result
    
    def fit(self, X):
        """
        self.y = p(y)
        self.x_y = p(x|y)
        self.x_y2 = self.x_y를 dictionary형태로 변환하여 저장한것
        """
        yRDD = sc.parallelize(X).map(lambda x: (0 if x[14] == ' <=50K' else 1,1)).reduceByKey(lambda x, y: x + y)
        y = sc.broadcast(yRDD.collect())
        yRDD = yRDD.mapValues(lambda x: x/X.shape[0])
        
        x_y = sc.parallelize(X).flatMap(lambda x: [((column, t, 0 if x[14] == ' <=50K' else 1),1) for column, t in enumerate(x[:-1])]).reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], x[1]/y.value[x[0][2]][1]))
        x_y2 = x_y.map(lambda pair: ((pair[0][0], pair[0][1]), (pair[0][2], pair[1]))).groupByKey().mapValues(list).collect()
        self.y = yRDD.collect()
        self.x_y = x_y.collect()
        self.x_y2 = dict(x_y2)
    
    def predict(self, X):
        """
        test set X가 들어온다면 해당 조건에 맞는 probability를 곱하여 ' <=50K' 인지 ' >50K'인지 결과 반환
        """
        result = []
        for x in X:
            p = [self.y[0][1], self.y[1][1]]
            for idx, key in enumerate(x):
                prob = self.find_prob((idx, key))
                p[0] *= prob[0]
                p[1] *= prob[1]
            if(p[0] >= p[1]):
                result.append(0)
            elif(p[0] < p[1]):
                result.append(1)
        
        self.predict = result
        return result
    
    def score(self, answer):
        """
        정답률 계산
        """
        cnt = 0
        for i in range(len(answer)):
            if(answer[i] == self.predict[i]):
                cnt += 1
        return cnt/len(answer) * 100

#### Quiz09 답안

In [8]:
def solution():
    X = create_dataset()
    kf = KFold(n_splits=10)
    
    score_sum = 0
    i = 1
    
    for train_index, test_index in kf.split(X):
        train, test = X[train_index], X[test_index]
        X_test = test[:,:-1]
        answer = [0 if x == ' <=50K' else 1 for x in test[:,14]]

        naive = NaiveBayes()
        naive.fit(train)
        predict = naive.predict(X_test)
        score = naive.score(answer)
        print("round {}: {}".format(i,score))

        score_sum += score
        i += 1

    print("====================================================")
    
    score_average = score_sum / 10
    print(score_average)

In [9]:
solution()

round 1: 79.33681301811482
round 2: 80.2272029474977
round 3: 80.98894348894349
round 4: 80.497542997543
round 5: 80.62039312039312
round 6: 81.51105651105651
round 7: 80.98894348894349
round 8: 80.8046683046683
round 9: 80.4054054054054
round 10: 80.12899262899262
80.55099619115585
